In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score


In [3]:

# Load and preprocess dataset
data = pd.read_csv('Food_Delivery_Times.csv')


In [4]:

# Drop rows with missing target and fill other missing values
data = data.dropna(subset=['Delivery_Time_min'])
data = data.fillna(method='ffill')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_5288\281867332.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


In [5]:

# Encode categorical features
categorical_cols = ['Weather', 'Traffic_Level', 'Time_of_Day', 'Vehicle_Type']
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)


In [6]:

# Features and Target
X = data.drop(columns=['Order_ID', 'Delivery_Time_min'])
y = data['Delivery_Time_min']


In [7]:

# Binarize target for classification purposes
y_binary = (y > y.median()).astype(int)


In [8]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)


In [9]:
# Bat Algorithm Parameters
num_bats = 10
max_iter = 10
loudness = 0.6
pulse_rate = 0.5
frequency_min, frequency_max = 0, 2

In [10]:
# Fitness Function
def fitness_function(solution):
    selected_features = np.where(solution > 0.5)[0]
    if len(selected_features) == 0:
        return 0, 0, 0, 0  # Return 0 for accuracy, precision, recall, and F1 if no features are selected

    # Train model on selected features
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train.iloc[:, selected_features], y_train)
    preds = model.predict(X_test.iloc[:, selected_features])
    preds_binary = (preds > 0.5).astype(int)

    # Calculate metrics
    accuracy = accuracy_score(y_test, preds_binary)
    precision = precision_score(y_test, preds_binary)
    recall = recall_score(y_test, preds_binary)
    f1 = f1_score(y_test, preds_binary)

    return accuracy, precision, recall, f1


In [11]:
# Initialize bats
positions = np.random.rand(num_bats, X.shape[1])  # Random positions in [0,1]
velocities = np.random.uniform(-1, 1, (num_bats, X.shape[1]))  # Random velocities
best_global = positions[np.random.randint(0, num_bats)]  # Randomly select a bat
best_global_accuracy, best_global_precision, best_global_recall, best_global_f1 = fitness_function(best_global)


In [12]:
# Bat Algorithm
for t in range(max_iter):
    for i in range(num_bats):
        # Calculate frequency and update velocity and position
        frequency = frequency_min + (frequency_max - frequency_min) * np.random.rand()
        velocities[i] += (positions[i] - best_global) * frequency
        positions[i] = np.clip(positions[i] + velocities[i], 0, 1)  # Ensure positions stay in range

        # Local search
        if np.random.rand() > pulse_rate:
            positions[i] = np.clip(best_global + np.random.normal(0, 0.1, size=X.shape[1]), 0, 1)

        # Evaluate fitness
        accuracy, precision, recall, f1 = fitness_function(positions[i])
        if accuracy > best_global_accuracy and np.random.rand() < loudness:
            best_global = positions[i]
            best_global_accuracy, best_global_precision, best_global_recall, best_global_f1 = accuracy, precision, recall, f1

    # Update loudness and pulse rate
    loudness = max(0.1, loudness * 0.95)
    pulse_rate = min(1.0, pulse_rate * 1.05)

    # Print progress
    print(f"Iteration {t + 1}:")
    print(f"   Best Accuracy = {best_global_accuracy:.4f}")
    print(f"   Best Precision = {best_global_precision:.4f}")
    print(f"   Best Recall = {best_global_recall:.4f}")
    print(f"   Best F1 Score = {best_global_f1:.4f}")
    selected_features = np.where(best_global > 0.5)[0]
    print("   Selected Features:", X.columns[selected_features].tolist())
    print()



Iteration 1:
   Best Accuracy = 0.8733
   Best Precision = 0.8786
   Best Recall = 0.8542
   Best F1 Score = 0.8662
   Selected Features: ['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs', 'Weather_Snowy', 'Traffic_Level_Medium', 'Time_of_Day_Evening', 'Time_of_Day_Morning']

Iteration 2:
   Best Accuracy = 0.8733
   Best Precision = 0.8786
   Best Recall = 0.8542
   Best F1 Score = 0.8662
   Selected Features: ['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs', 'Weather_Snowy', 'Traffic_Level_Medium', 'Time_of_Day_Evening', 'Time_of_Day_Morning', 'Time_of_Day_Night']

Iteration 3:
   Best Accuracy = 0.8767
   Best Precision = 0.8849
   Best Recall = 0.8542
   Best F1 Score = 0.8693
   Selected Features: ['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs', 'Weather_Foggy', 'Weather_Windy', 'Traffic_Level_Medium', 'Time_of_Day_Evening', 'Time_of_Day_Morning', 'Time_of_Day_Night', 'Vehicle_Type_Scooter']

Iteration 4:
   Best Accuracy = 0.8767
  

In [13]:


# Final Results
print("Final Results:")
print(f"Best Accuracy: {best_global_accuracy:.4f}")
print(f"Best Precision: {best_global_precision:.4f}")
print(f"Best Recall: {best_global_recall:.4f}")
print(f"Best F1 Score: {best_global_f1:.4f}")
selected_features = np.where(best_global > 0.5)[0]
print("Selected Features:", X.columns[selected_features].tolist())


Final Results:
Best Accuracy: 0.8767
Best Precision: 0.8849
Best Recall: 0.8542
Best F1 Score: 0.8693
Selected Features: ['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs', 'Weather_Foggy', 'Weather_Windy', 'Traffic_Level_Medium', 'Time_of_Day_Evening', 'Time_of_Day_Morning', 'Time_of_Day_Night', 'Vehicle_Type_Car', 'Vehicle_Type_Scooter']
